# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  
    """
    构造一个PyTorch数据迭代器
        data_arrays: 包含特征和标签的元组/列表，如 (features, labels)
        batch_size: 每个批次的数据量
        is_train: 是否为训练模式（影响是否打乱数据）
    """
    # TensorDataset 将特征和标签打包成一个数据集，data_arrays 是解包操作
    # 等价于 TensorDataset(features, labels)，这样每个样本都是 (特征, 标签) 的配对
    dataset = data.TensorDataset(*data_arrays)
    # DataLoader 是核心迭代器，提供批量加载功能，shuffle=is_train：训练时打乱数据顺序，测试时保持原顺序
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
"""
iter(data_iter): 将DataLoader转为迭代器
next(): 获取第一个批次的数据
返回结果类似：[tensor([...]), tensor([...])]，第一个是特征，第二个是标签
在训练循环中，for X, y in data_iter: 会自动遍历所有批次
"""
next(iter(data_iter))

[tensor([[-0.0572,  0.7032],
         [ 0.4396, -0.2956],
         [-0.5142,  0.2402],
         [-0.2134,  0.7916],
         [-0.5279, -0.5017],
         [-0.5091,  0.1502],
         [-0.4009,  1.8969],
         [-1.4191,  1.9877],
         [-0.4310,  0.3707],
         [-0.7785,  0.0942]]),
 tensor([[ 1.7047],
         [ 6.0855],
         [ 2.3521],
         [ 1.0722],
         [ 4.8392],
         [ 2.6702],
         [-3.0385],
         [-5.3891],
         [ 2.0902],
         [ 2.3309]])]

使用框架的预定义好的层

In [3]:
# nn是PyTorch的神经网络库，包含所有模型构建块
from torch import nn
"""
nn.Sequential：按顺序堆叠层的容器，数据会依次通过各层
nn.Linear(2, 1)：全连接层（线性层），执行 y = XWᵀ + b 运算
2：输入特征维度（每个样本有2个特征）
1：输出维度（输出一个标量值）
架构：单层线性模型
参数量：3个（2个权重 + 1个偏置）
数学表示：
输出 = 特征₁×w₁ + 特征₂×w₂ + b
"""
net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [4]:
"""
1、初始化权重（正态分布）
net[0]：访问 Sequential 中的第 0 层（即 nn.Linear(2, 1)）
.weight：获取该层的权重矩阵（形状：[1, 2]）
.data：直接操作底层张量（绕过梯度追踪）
.normal_(0, 0.01)：用正态分布随机填充，均值为0，标准差为0.01
下划线 _ 表示原地操作（in-place），直接修改张量值
"""
net[0].weight.data.normal_(0, 0.01)
# 2、初始化偏置（全零）
# .bias  ：获取该层的偏置向量（形状：[1]）；.fill_(0)  ：将所有元素填充为0
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数，$MSE=\frac{1}{n}\sum_{i=1}^{n}(y_{pred_{i}}-y_{true_{i}})^{2}$

In [5]:
loss = nn.MSELoss()

实例化一个`SGD`实例，$\theta=\theta-lr\times\bigtriangledown_{\theta}Loss$

In [6]:
# net.parameters() ：获取模型中所有需要训练的参数（权重和偏置）
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [7]:
"""
前向-损失-反向-更新
for epoch in range(num_epochs):
    for 每个批次 (X, y):
        1. 预测 → net(X)
        2. 算损失 → loss()
        3. 清梯度 → zero_grad()
        4. 反向传播 → backward()
        5. 更新参数 → step()
    # 一轮结束，评估并打印
"""
# 1. 设置训练轮数
num_epochs = 3
# 2. 外层循环：轮次迭代
for epoch in range(num_epochs):
    # 3. 内层循环：批量训练
    """
    data_iter 是之前创建的DataLoader，每次返回一个批次的数据
    X： batch_size个样本的特征
    y：对应的标签
    """
    for X, y in data_iter:
        # 4. 前向传播 + 计算损失
            #net(X)：将特征输入网络，得到预测值；loss()：计算预测值与真实值的均方误差
        l = loss(net(X) ,y)
        # 5. 清空梯度
        trainer.zero_grad()
        # 6. 反向传播：自动计算损失对所有权重的梯度；梯度存储在weight.grad和bias.grad中
        l.backward()
        # 7. 更新参数，w = w - lr * w.grad
        trainer.step()
    # 8. 评估模型，在torch.no_grad()下执行（避免计算测试梯度）
    l = loss(net(features), labels)
    # 9. 打印日志
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000224
epoch 2, loss 0.000106
epoch 3, loss 0.000105


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [8]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0002, 0.0009])
b的估计误差： tensor([0.0003])
